In [ ]:
#CNN모델, 그리고 머신러닝을 활용하기 위해 tensorflow를 사용
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import numpy as np
from sklearn.utils import shuffle

In [ ]:
#디스코드봇의 운영을 위해 필요한 입력함수
import discord
from discord.ext import commands
import asyncio
import nest_asyncio
import pandas as pd
#이미지를 배열로 저장
import numpy as np
#검열, 허용범위의 이미지를 가져오기 위해 필요 
from PIL import Image
import io

In [ ]:
#검열 데이터(0)
censored_images=["censored1(Danger).jpg", "censored2(Danger).png","censored3(Danger).jpg","censored4(Danger).jpg","censored5(Danger).jpg","censored6(danger).jpg","censored7(Danger).png","censored8(Danger).jpg","censored9(Danger).jpg","censored10(Danger).jpg"]

#허용범위(1)
valid_images=["valid1.png", "valid2.png", "valid3.png", "valid4.png", "valid5.jpg","lego.png","miku.png","bluecookie.jpg","number.jpg","roka.jpg","bone.png","kabuki.jpg","kabuki2.jpg"]

#지도학습에 쓰인 사진 데이터들(검열데이터, 허용범위 데이터)
def load_and_process_images(image_list, label):
    data = [] 
    labels = []
    for img_path in image_list:
        img = Image.open(img_path).resize((150, 150)).convert("RGB")  # RGB 변환
        img_array = np.array(img) / 255.0  # 정규화
        data.append(img_array)
        labels.append(label)  
    return np.array(data, dtype=np.float32), np.array(labels, dtype=np.int32)

# 단일 이미지(바이트 데이터)를 CNN 입력 형식으로 변환하는 함수
def preprocess_image(image_bytes):
    image = Image.open(io.BytesIO(image_bytes)).resize((150, 150)).convert("RGB")  # RGB 변환
    image = np.array(image) / 255.0  # 정규화
    image = np.expand_dims(image, axis=0)  # 배치 차원 추가 (1, 150, 150, 3)
    return image.astype(np.float32)  # TensorFlow 호환을 위해 float32로 변환

# 검열된 이미지 (라벨 0) 로드
censored_data, censored_labels = load_and_process_images(censored_images, 0)

# 허용된 이미지 (라벨 1) 로드
valid_data, valid_labels = load_and_process_images(valid_images, 1)

# 데이터 합치기
X = np.concatenate((censored_data, valid_data), axis=0)
y = np.concatenate((censored_labels, valid_labels), axis=0)

# y의 데이터값 증강 (차원 변경)
y = y.reshape(-1, 1)

# 데이터 섞기
X, y = shuffle(X, y, random_state=42)


In [ ]:
print(X.shape)

In [ ]:
print(y.shape)

In [ ]:
print(X.shape, y.shape)

In [ ]:
# CNN 모델 정의 (이진 분류)
#Sequential ->keras에서 신경망 모델을 만드는 클래스 
# 데이터 로드
censored_data, censored_labels = load_and_process_images(censored_images, 0)  # 검열(0)
valid_data, valid_labels = load_and_process_images(valid_images, 1)  # 허용(1)

# 데이터 합치기
X = np.concatenate((censored_data, valid_data), axis=0)
y = np.concatenate((censored_labels, valid_labels), axis=0)

# 데이터 섞기
X, y = shuffle(X, y, random_state=42)

# CNN 모델 정의 (이진 분류)
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(128,(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')  # 이진 분류
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 데이터 증강 적용(데이터 불균형 해소 목적) 
datagen = ImageDataGenerator(rotation_range=20, horizontal_flip=True)
train_generator = datagen.flow(X, y, batch_size=32)

# 모델 학습
model.fit(train_generator, epochs=10)

# 모델 저장
model.save("image_filter_model.h5")


In [ ]:
nest_asyncio.apply()

# 디스코드 봇 토큰 (여기에 자신의 봇 토큰 입력)
TOKEN = 'your token'
CHANNEL_ID = 'your server ID'

# 저장된 CNN 모델 로드
model = tf.keras.models.load_model("image_filter_model.h5")

bot = commands.Bot(command_prefix="/", intents=discord.Intents.all())

# 이미지 전처리 함수 (CNN 입력 형식으로 변환)
def preprocess_image(image_bytes):
    image = Image.open(io.BytesIO(image_bytes)).resize((150, 150)).convert("RGB")  # RGB 변환
    image = np.array(image) / 255.0  # 정규화

    # 잘못된 차원 확장 제거
    image = np.expand_dims(image, axis=0)  # 배치 차원 추가 (1, 150, 150, 3) - ✅ 올바른 방식
    print(f"📌 수정된 입력 이미지 shape: {image.shape}")  # 디버깅용

    return image.astype(np.float32)  # TensorFlow 호환을 위해 float32로 변환

@bot.event
async def on_ready():
     print("Bot ready!")
    
@bot.event
async def on_message(message):
    if message.author == bot.user:
        return

    for attachment in message.attachments:
        if attachment.content_type.startswith('image'):
            image_bytes = await attachment.read()
            image = preprocess_image(image_bytes)

            print(f"📌 입력 이미지 shape: {image.shape}")  # 예상: (1, 150, 150, 3)

            prediction = model.predict(image)
            predicted_label = int(prediction[0][0] > 0.5)  # 0: 검열됨, 1: 정상

            if predicted_label == 0:
                await message.delete()
                await message.channel.send(f"🚨 {message.author.mention}, 유해한 이미지가 감지되어 삭제되었습니다.")
                print(f"❌ 유해 이미지 삭제됨: {message.author}가 업로드한 이미지")
            else:
                print(f"✅ 정상 이미지: {message.author}가 업로드한 이미지")


async def start_bot():
    await bot.start(TOKEN)

asyncio.run(start_bot())